In [ ]:
from fastlane_bot.tools.cpc import CPCContainer, ConstantProductCurve as CPC, CurveBase
from fastlane_bot.tools.optimizer import MargPOptimizer, PairOptimizer
from fastlane_bot.testing import *
# from flbtools.cpc import CPCContainer, ConstantProductCurve as CPC, CurveBase
# from flbtools.optimizer import MargPOptimizer, PairOptimizer
# from flbtesting import *

from math import sqrt
from copy import deepcopy
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(CPCContainer))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(CPC))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(MargPOptimizer))
print("{0.__name__} v{0.__VERSION__} ({0.__DATE__})".format(PairOptimizer))

plt.style.use('seaborn-v0_8-dark')
plt.rcParams['figure.figsize'] = [12,6]
# from fastlane_bot import __VERSION__
# require("3.0", __VERSION__)

# Optimization Methods [NBTest053]

Note: using an existing CPCContainer object `CC`, the curves can be extracted as dict using the command below

    CURVES = [c.asdict() for c in CC]
    

The below three curves are one POL curve (extremely levered; it is originally fixed price) and two Uniswap v3 curves. On those curves the high dimensional gradient descent algo fails because it ends up on a plateau.

In [ ]:
CURVES = [

# POL Curve
{
  'k': 6.157332844764952e+20,
  'x': 615733222.5892723,
  'x_act': 0,
  'y_act': 100000.0,
  'alpha': 0.5,
  'pair': 'WETH/DAI', # WETH-6Cc2/DAI-1d0F
  'cid': '0x33ed',
    #     0x33ed451d5c7b7a76266b8cdfab030f6de8143fcfbdcd08dabeed0de8d684b4de
  'fee': 0.0,
  'descr': 'bancor_pol DAI-1d0F/ETH-EEeE 0.000',
  'constr': 'carb',
  'params': {'exchange': 'bancor_pol',
   'tknx_dec': 18,
   'tkny_dec': 18,
   'tknx_addr': '0x6B175474E89094C44Da98b954EedeAC495271d0F',
   'tkny_addr': '0xEeeeeEeeeEeEeeEeEeEeeEEEeeeeEeeeeeeeEEeE',
   'blocklud': 18121620,
   'y': 100000.0,
   'yint': 100000.0,
   'A': 0,
   'B': 40.29987368093254,
   'pa': 1624.0799811071013,
   'pb': 1624.0799811071013}},
 
# Uniswap v3 Curve 1
 {
  'k': 1147678924959.0112,
  'x': 42728400.31211105,
  'x_act': 7575.552803896368,
  'y_act': 8.665306719478394,
  'alpha': 0.5,
  'pair': 'DAI/WETH', # DAi-1d0F/WETH-6Cc2
  'cid': '0xb1d8',
    #     0xb1d8cd62f75016872495dae3e19d96e364767e7d674488392029d15cdbcd7b34',
  'fee': 0.0005,
  'descr': 'uniswap_v3 DAI-1d0F/WETH-6Cc2 500',
  'constr': 'pkpp',
  'params': {'exchange': 'uniswap_v3',
   'tknx_dec': 18,
   'tkny_dec': 18,
   'tknx_addr': '0x6B175474E89094C44Da98b954EedeAC495271d0F',
   'tkny_addr': '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2',
   'blocklud': 18121789,
   'L': 1071297.7760450225}},
    

# Uniswap v3 Curve 2
{
  'k': 1541847511355.546,
  'x': 49517090.33542573,
  'x_act': 99496.94394361228,
  'y_act': 30.763865271412214,
  'alpha': 0.5,
  'pair': 'DAI/WETH', # DAi-1d0F/WETH-6Cc2
  'cid': '0xae2b',
      #  '0xae2b487dff467a33b88e5a4e6874f91ee212886979f673dd18d3e0396862112f',
  'fee': 0.003,
  'descr': 'uniswap_v3 DAI-1d0F/WETH-6Cc2 3000',
  'constr': 'pkpp',
  'params': {'exchange': 'uniswap_v3',
   'tknx_dec': 18,
   'tkny_dec': 18,
   'tknx_addr': '0x6B175474E89094C44Da98b954EedeAC495271d0F',
   'tkny_addr': '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2',
   'blocklud': 18121689,
   'L': 1241711.5250151888}}
]
CC = CPCContainer.from_dicts(CURVES)

Those are starting prices consistent with those curves.

In [ ]:
PRICES = {
    'DAI':  0.0006286424878113893, 
    'WETH': 1,
}
PRICE0 = PRICES["WETH"]/PRICES["DAI"]
PRICE0

`CC0` is meant to be a variation of `CC` / `CURVES` that converges, but it does not seem to work. Not super high priority to find one.

In [ ]:
c0 = CC[0]

In [ ]:
xyp = c0.xyfromp_f(1592, withunits=True, ignorebounds=True)
CURVES0 = deepcopy(CURVES)
CURVES0[0]["x"] = xyp[0]
CC0 = CPCContainer.from_dicts(CURVES0)
xyp

In [ ]:
CC[0].p, CC0[0].p

This is a very simple set of unlevered curver where convergence should never be a problem.

In [ ]:
CCul = CPCContainer([
    CPC.from_pk(p=1500, k=1500*100, pair="WETH/DAI", cid="c1500"),
    CPC.from_pk(p=1600, k=1600*100, pair="WETH/DAI", cid="c1600")
])

In [ ]:
cnorm = CPC.from_pk(p=PRICE0, k=PRICE0*c0.x, pair="WETH/DAI", cid="normalizer")
CCn = CPCContainer([c for c in CC]+[cnorm])

In [ ]:
c0.x, c0.tknx

In [ ]:
#help(CPC)

## MargPOptimizer current
### Setup

In [ ]:
#help(MargPOptimizer)

### Unlevered curves `CCul`

In [ ]:
Oul = MargPOptimizer(curves=CCul)
assert len(Oul.curves) == len(CCul)

In [ ]:
r = Oul.optimize("WETH")
assert r.error is None
assert r.method == "margp"
assert r.targettkn == "WETH"
assert r.tokens_t == ('DAI',)
assert r.dtokens["WETH"] < 0
assert iseq(r.result, -0.005204267821271813)
assert iseq(r.p_optimal_t[0], 0.0006449934107164284)
assert iseq(r.dtokens_t[0], -4.737194103654474e-08)
r

the original curves are 1500 and 1600, so ~1550 is right in the middle

In [ ]:
assert iseq(1/r.p_optimal_t[0], 1550.4034357331604)
1/r.p_optimal_t[0]

this process converged -- the aggregate change in DAI amount < 1e-5

In [ ]:
assert abs(r.dtokens["DAI"] < 1e-5)
assert r.dtokens["WETH"] < 0
assert iseq(r.dtokens["WETH"], -0.005204267821271813)
r.dtokens

there is some trading going on

In [ ]:
v = r.dxvecvalues(asdict=True)
assert iseq(v["c1500"]["DAI"], 249.9349296963901)
assert iseq(v["c1600"]["WETH"], 0.15868818867025603)
v

### Normalized curves `CCn`

In [ ]:
On  = MargPOptimizer(curves=CCn)
assert len(On.curves) == len(CC)+1

In [ ]:
r = On.optimize("WETH")
assert r.error is None
assert r.method == "margp"
assert r.targettkn == "WETH"
assert r.tokens_t == ('DAI',)
assert r.dtokens["WETH"] < 0
assert iseq(r.result, -1.244345098228223)
assert iseq(r.p_optimal_t[0], 0.00062745798800732)
assert iseq(r.dtokens_t[0], -1.9371509552001953e-06, eps=0.1)
# assert iseq(r.dtokens_t[0], -1.9371509552001953e-06, eps=0.01)     # FAILS ON GITHUB
# assert iseq(r.dtokens_t[0], -1.9371509552001953e-06, eps=0.001)    # FAILS ON GITHUB
# assert iseq(r.dtokens_t[0], -1.9371509552001953e-06, eps=0.0001)   # FAILS ON GITHUB
r

the original curves are 1500 and 1600, so ~1550 is right in the middle

In [ ]:
assert iseq(1/r.p_optimal_t[0], 1593.7322005825413, eps=0.001)
1/r.p_optimal_t[0]

this process converged -- the aggregate change in DAI amount < 1e-5

In [ ]:
assert abs(r.dtokens["DAI"] < 1e-5)
assert r.dtokens["WETH"] < 0
assert iseq(r.dtokens["WETH"], -1.244345098228223)
r.dtokens

there is some trading going on

In [ ]:
v = r.dxvecvalues(asdict=True)
v

### Succeeding optimization process `CC0` [TODO]

note: the parameters still don't allow for convergence!

In [ ]:
O0  = MargPOptimizer(curves=CC0)
assert len(O0.curves) == len(CC)

In [ ]:
r = O0.optimize("WETH")
assert r.error is None
assert r.method == "margp"
assert r.targettkn == "WETH"
assert r.tokens_t == ('DAI',)
assert iseq(r.result, -39.42917199089061)
assert iseq(r.p_optimal_t[0], 0.0006273686958774544)
assert iseq(r.dtokens_t[0], 62760.13561845571)
r

In [ ]:
r.dtokens

In [ ]:
r.dxvecvalues(asdict=True)

### Failing optimization process `CC`

In [ ]:
O   = MargPOptimizer(curves=CC)
assert len(O.curves) == len(CC)

In [ ]:
r = O.optimize("WETH")
assert r.error is None
assert r.method == "margp"
assert r.targettkn == "WETH"
assert r.tokens_t == ('DAI',)
assert iseq(r.result, 22.14415018604268)
assert iseq(r.p_optimal_t[0], 0.0006273686958774544)
assert iseq(r.dtokens_t[0], -37239.86438154429)
r

Here we show that the final price is not the same as the initial one, but also not totally crazy (this calculation has not converged but is stuck on a plateau)

In [ ]:
PRICES, r.p_optimal

In [ ]:
1/r.p_optimal_t[0], PRICES["WETH"]/PRICES["DAI"]

The `result` is the amount of target token extracted. Note that this assumes that the algo has converged which it has not in this case. The `dtokens` property shows the _aggregate_ change in tokens, and it _should_ be zero for everything but the target token WETH which is not the case here.

In [ ]:
assert r.result == r.dtokens["WETH"]
r.result

In [ ]:
r.dtokens

`dxdyvalues` and `dxvecvalues` show the changes of the respective curves. For standard two-asset curves they are equivalent, just in a different format; for three+ asset curves only dxvecvalues is defined

In [ ]:
r.dxdyvalues(asdict=True)

In [ ]:
r.dxvecvalues(asdict=True)

This shows that the algorithm **has not converged** -- this number (the net flow of DAI; note that the target token here is WETH) should be zero!

In [ ]:
s_DAI = sum(x["DAI"] for x in r.dxvecvalues(asdict=True).values())
assert iseq(s_DAI, r.dtokens["DAI"])
s_DAI

This number is not expected to be zero as the profit is being extracted in WETH

In [ ]:
s_WETH = sum(x["WETH"] for x in r.dxvecvalues(asdict=True).values())
assert iseq(s_WETH, r.dtokens["WETH"])
s_WETH

## PairOptimizer vs MarpP
### Setup

### Unlevered curves `CCul`

In [ ]:
Oul = PairOptimizer(curves=CCul)
Oul_mp = MargPOptimizer(curves=CCul)
assert len(Oul.curves) == len(CCul)

Unlevered curves converged nicely in the margp (gradient descent) optimizer, and they are converging nicely here; the results are very close together (better than 1e-5)

In [ ]:
r   = Oul.optimize("WETH")
rmp = Oul_mp.optimize("WETH")
assert r.error is None
assert rmp.error is None
assert r.method   == "margp-pair"
assert rmp.method == "margp"
assert r.targettkn == "WETH" 
assert rmp.targettkn == "WETH"
assert r.tokens_t == ('DAI',)
assert rmp.tokens_t == ('DAI',)
assert r.dtokens["WETH"] < 0
assert rmp.dtokens["WETH"] < 0
assert iseq(r.p_optimal_t[0], 0.0006449934107144566)
assert iseq(rmp.p_optimal_t[0], 0.0006449934107164284)
assert r.result/rmp.result-1 < 1e-5
r, rmp, r.result/rmp.result-1

It is notable that the bisection algorithm is **six times slower** than the gradient descent

In [ ]:
r.time/rmp.time

the optimal price here is very very close: 1e-12

In [ ]:
assert r.p_optimal_t[0]/rmp.p_optimal_t[0]-1 < 1e-8
r.p_optimal_t[0]/rmp.p_optimal_t[0]-1

Here we show that (a) the DAI transfer is de-minimis and close enough to zero, and more importantly, that (b) both our methods give essentially the same result as to how much ETH can be obtained from the arb

In [ ]:
assert r.dtokens["DAI"] < 1e-5
assert rmp.dtokens["DAI"] < 1e-5
assert r.dtokens["WETH"]/rmp.dtokens["WETH"]-1 < 1e-5
r.dtokens, rmp.dtokens, r.dtokens["WETH"]/rmp.dtokens["WETH"]-1

### Normalized curves `CCn`

In [ ]:
On = PairOptimizer(curves=CCn)
On_mp = MargPOptimizer(curves=CCn)
assert len(On.curves) == len(CC)+1

In [ ]:
r = On.optimize("WETH")
rmp = On_mp.optimize("WETH")
assert r.error is None
assert rmp.error is None
assert r.method   == "margp-pair"
assert rmp.method == "margp"
assert r.targettkn == "WETH" 
assert rmp.targettkn == "WETH"
assert r.tokens_t == ('DAI',)
assert rmp.tokens_t == ('DAI',)
assert r.dtokens["WETH"] < 0
assert rmp.dtokens["WETH"] < 0
assert iseq(r.p_optimal_t[0], 0.0006274579880072543)
assert iseq(rmp.p_optimal_t[0], 0.00062745798800732)
assert r.result/rmp.result-1 < 1e-5
r, rmp, r.result/rmp.result-1

### Succeeding optimization process `CC0` [TODO]

In [ ]:
O0  = PairOptimizer(curves=CC0)
O0_mp  = MargPOptimizer(curves=CC0)
assert len(O0.curves) == len(CC) # sic

In [ ]:
r = O0.optimize("WETH")
r

### Optimization process `CC` (fails in full margp)

In [ ]:
O = PairOptimizer(curves=CC)
O_mp   = MargPOptimizer(curves=CC)
assert len(O.curves) == len(CC)

In [ ]:
r = O.optimize("WETH")
rmp = O_mp.optimize("WETH")
assert r.error is None
assert rmp.error is None
assert r.method   == "margp-pair"
assert rmp.method == "margp"
assert r.targettkn == "WETH" 
assert rmp.targettkn == "WETH"
assert r.tokens_t == ('DAI',)
assert rmp.tokens_t == ('DAI',)
assert r.dtokens["WETH"] < 0
assert not rmp.dtokens["WETH"] < 0 # FAILS!
assert iseq(r.p_optimal_t[0], 0.0006157332379890538)
assert iseq(rmp.p_optimal_t[0], 0.0006273686958774544)
assert r.result/rmp.result-1 < 1e-5
r, rmp, r.result/rmp.result-1

This now converges fine (note as we see below we need an eps parameter of about 1e-10, and also not that we can't go much higher because in this case it gets stuck, probably because of float precision.

In [ ]:
r.dtokens, r.dtokens["WETH"]*PRICE0

We see that accuracy at eps=1e-6 is not that great, but at 1e-10 it is very good; also it seems that by and large the runtime does not really depend on the precision parameter here, so we go for 1e-10 throughout [not you can't go for higher precision as it then never returns, probably because of float accuracy issues]

In [ ]:
r06 = O.optimize("WETH", params={"eps":1e-6})
r08 = O.optimize("WETH", params={"eps":1e-8})
r10 = O.optimize("WETH", params={"eps":1e-10})
r06.dtokens, r08.dtokens, r10.dtokens

In [ ]:
[r10.time/r06.time, r08.time/r06.time]

## MargPOptimizer new TODO

this is still on the todo lost, but does not have high priority; the new margp optimizer will have a different convergence criterium [p ~ 0 rather than d log p ~ 0]. This will not help in terms of convergence on a plateau -- a gradient algorithm can not recover from f'(x) = 0 -- but it will allow identifying instances of non convergence.

### Setup

In [ ]:
pass

In [ ]:
# Oul = PairOptimizer(curves=CCul)
# On  = PairOptimizer(curves=CCn)
# O0  = PairOptimizer(curves=CC0)
# O   = PairOptimizer(curves=CC)
# assert len(On.curves) == len(CC)+1
# assert len(O0.curves) == len(CC)
# assert len(O.curves) == len(CC)

### Unlevered curves `CCul`

### Normalized curves `CCn`

### Succeeding optimization process `CC0` [TODO]

### Failing optimization process `CC`

## Charts [NOTEST]

In [ ]:
CC.plot()

In [ ]:
CCul.plot()

In [ ]:
CC0.plot()

In [ ]:
CCn.plot()